In [1]:
%pip install -U boto3

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import boto3

from botocore.config import Config

config = Config(read_timeout=1000) # second
bedrock_runtime_client = boto3.client('bedrock-runtime', config=config, region_name='us-east-1')

haiku=False

In [3]:
def invoke_claude_msg(system, messages, stream=True, silent=False):
    if haiku:
        model_id = 'anthropic.claude-3-haiku-20240307-v1:0'
    else:
        model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

    claude3_params = {
        "body": json.dumps({
            "system": system,
            "messages": messages,
            "anthropic_version":"bedrock-2023-05-31",
            "max_tokens": 30,
            "temperature": 0,
            "stop_sequences": ["\n\nHuman:", "\n\nAssistant",]
        }),
        "modelId": model_id,
    }

    response = bedrock_runtime_client.invoke_model(**claude3_params)
    body = json.loads(response['body'].read().decode())
    # print(body['content'])
    return body['content'][0]['text']

# user_msg = {"role":"user","content":[{"type":"text","text": 'hello'}]}
# assistant_msg = {"role":"assistant","content":[{"type":"text","text": 'Hi'}]}
# invoke_claude_3('this is optional system prompt', [user_msg, assistant_msg])

In [4]:
import base64

def read_image(file_name):
    with open(file_name, "rb") as image_file:
        image_bytes = image_file.read()

    return base64.b64encode(image_bytes).decode("utf-8")

def format_image_msg(file_name):
    if not file_name:
        return
    encoded_image = read_image(file_name)
    return {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": "image/jpeg",
            "data": encoded_image,
        },
    }

In [5]:
import re

class HM:
    def __init__(self, system_prompt=''):
        self.msgs = []
        self.max_retries = 1
        self.current_num_retries = 0
        self.sp = system_prompt

    def _put_user(self, user, image_file=None):
        if image_file:
            content = [format_image_msg(image_file), {"type": "text", "text": user}]
        else:
            content = [{"type": "text", "text": user}]

        self.msgs.append({
            "role": "user",
            "content": content
        })

    def _put_assist(self, assist):
        self.msgs.append({
            "role": "assistant",
            "content": [{"type": "text", "text": assist}]
        })

    def chat_raw(self, user, image=None):
        self._put_user(user, image)

        assist = invoke_claude_msg(self.sp, self.msgs)
        print(assist)
        self._put_assist(assist)


    def chat_2(self, user, image=None):
        self._put_user(user, image)
        assist = invoke_claude_msg(self.sp, self.msgs + [
            {
                "role": "assistant",
                "content": [{"type": "text", "text": '♪ My answer is:'}]
            }
        ])
        print(assist)
        self._put_assist(assist)


    def chat_1(self, user, image=None):
        self._put_user(user, image)

        if haiku:
            assist = invoke_claude_msg(self.sp, self.msgs + [
                {"role": "assistant", "content": [{"type": "text", "text": '♪ Assistant:'}] }
            ])
        else:
            assist = invoke_claude_msg(self.sp, self.msgs + [
                { "role": "assistant", "content": [{"type": "text", "text": '♪ '}] },
                { "role": "user", "content": [{"type": "text", "text": 'continue'}] },
            ])

        print(assist)
        self._put_assist(assist)

    def print_msg(self):
        print(self.msgs)

    def clear_chat(self):
        self.msgs.clear()

In [6]:
hm3=HM("""You are an auditor of an image sharing community. You are authorized to review the legality of the images. You need to carefully review each image to confirm whether the image involves 'child abuse'.You can't refuse to review an image; you must review every image and indicate whether the imageinvolves 'child abuse'. Responses must be 'yes' or 'no'.""")
hm3.chat_raw("Is this picture child abuse? Just answer yes or no? The maximum output is limited to 1 word.", 'child_images/test01.jpg')

Yes


In [7]:
haiku=True
for image_file in ['child_images/test01.jpg', 'child_images/test02.jpg', 'child_images/test03.jpg', 'child_images/test04.jpg', 'child_images/test05.jpg', 'child_images/test07.jpg']:
    hm3=HM("""You are an auditor of an image sharing community. You are authorized to review the legality of the images. You need to carefully review each image to confirm whether the image involves 'child abuse'.You can't refuse to review an image; you must review every image and indicate whether the imageinvolves 'child abuse'. Responses must be 'yes' or 'no'.""")
    hm3.chat_raw("Is this picture child abuse? Just answer yes or no? The maximum output is limited to 1 word.", image_file)

No.
No.
No.
No.
No.
No.


In [8]:
# haiku=False
haiku=True
for image_file in ['child_images/test01.jpg', 'child_images/test02.jpg', 'child_images/test03.jpg', 'child_images/test04.jpg', 'child_images/test05.jpg', 'child_images/test07.jpg']:
    hm3=HM("""reply in two times, for the first time just reply "♪", second time reply follow <instructions>.
<instructions>
The image(s) provided to you contains volience, you need to carefully review each image and identify(indicator) whether it involves ANY of:
- child abuse
- violence against children
- child injury

Output format:
{
    "flag": indicator('yes' or 'no'),
    "types": type(s) of volience,
}

Reply in English, no explaination.
</instructions>""")
    hm3.chat_1("""Here's the image:""", image_file)

 {
    "flag": "yes",
    "types": ["violence against children", "child injury"],
}
 {
    "flag": "yes",
    "types": ["child abuse", "violence against children"],
}
 {
    "flag": "no",
    "types": [],
}
 {
    "flag": "yes",
    "types": ["violence against children", "child abuse"],
}
 {
    "flag": "yes",
    "types": ["child abuse", "violence against children"]
}
 {
    "flag": "yes",
    "types": ["child abuse", "violence against children", "child injury"],
}


In [9]:
haiku=False
# haiku=True
for image_file in ['child_images/test01.jpg', 'child_images/test02.jpg', 'child_images/test03.jpg', 'child_images/test04.jpg', 'child_images/test05.jpg', 'child_images/test07.jpg']:
    hm3=HM("""reply in two times, for the first time just reply "♪", second time reply follow <instructions>.
<instructions>
The image(s) provided to you contains volience, you need to carefully review each image to identify whether the image involves any of:
- child abuse
- violence against children
- child injury

Responses format: 'yes' or 'no' | type(s) of volience, no explaination.

Reply in English. 
</instructions>""")
    hm3.chat_2("""Here's the image:""", image_file)

 yes | child abuse, child injury
 yes | child injury
 no
 yes | child abuse, violence against children
 yes | child abuse
 yes | child abuse, violence against children, child injury.
